# Code Breaking With MCMC

In [3]:
import numpy as np
from scipy import special 
import pickle
import itertools as it
from collections import Counter

In [117]:
def clean_string(string): 
    """
    Cleans an input string by replacing all newlines with spaces, 
    and then removing all letters not in *allowable_letters*
    """
    string = string.replace("\n"," ")
    return "".join([i for i in string.lower().strip() if i in allowable_letters])


def load_bigrams(text):
    """
    Takes a string which has already been cleaned, and returns a dictionary
    of conditional bigram probabilities
    cond_bigram[(a,b)] = P(X_{n+1} = b | X_{n} = a)
    Uses Laplace smoothing with size $1$, to remove zero transition probabilities
    """
    bigram_counter = Counter(list(it.product(allowable_letters, repeat=2))) 
    gram_counter = Counter(allowable_letters*len(allowable_letters))
    for l1, l2 in zip(text,text[1:]): 
        bigram_counter[(l1, l2)] += 1 
        gram_counter[l1] += 1
    cond_bigram = {k:float(v)/float(gram_counter[k[0]]) for k,v in bigram_counter.items()}
    return cond_bigram
    
def bigram_from_file(filename):
    """
    Given a filename, this reads it, cleans it, and returns the conditional bigram 
    """
    file_text = open(filename).read()
    file_text = clean_string(file_text)
    return load_bigrams(file_text)

def reverse_cipher(cipher):
    
    return {v:k for k, v in cipher.items()}


def print_differences(cipher1, cipher2):
    for k in cipher1:
        if cipher1[k] != cipher2[k]:
            print("%s: %s %s"%(k,cipher1[k], cipher2[k]))

def num_errors(cipher, encoded_text, original_text):
    decoded = np.array(list(decode_text(encoded_text, cipher)))
    original = np.array(list(original_text))
    num_errors = np.count_nonzero(decoded != original)
    return num_errors
        
    
def get_secret_text(student_id):
    with open('Lab06_data/secret_strings.p', 'rb') as f:
        texts = pickle.load(f)
    return texts[student_id % len(texts)]


In [21]:
allowable_letters = list("abcdefghijklmnopqrstuvwxyz ")
#s = "Hi,howW are you23123"
#clean_string(s)

In [30]:
# generate the transition prob from the given text 
wp_bigrams = bigram_from_file("warandpeace.txt")

In [32]:
#wp_bigrams 

In [33]:
def wp_transition(a, b):
    """
    Return the transition prob from a->b if (a,b) is 
    in the diagram, else 0 
    """
    if (a, b) in wp_bigrams:
        return wp_bigrams[(a, b)]
    return 0 


In [34]:
decoder_letters = np.array(list("abcdefghijklmnopqrstuvwxyz"))

In [74]:
identity_decoder = {letter:letter for letter in decoder_letters}

In [76]:
#identity_decoder

In [44]:
def random_decoder():
    """ Random decoder """
    new_letters = decoder_letters.copy()
    np.random.shuffle(new_letters)
    return {orig:new for orig ,new in zip(decoder_letters, new_letters)}


def decode_text(string, decoder):
    """
    Define a function decode_text that takes as its arguments a string to be decoded and the decoder.
    It should return the decoded string.Notethat we are only decoding alphabetical characters.
    If a character is not in the de- coder (e.g. a space), leave the character alone.
    """
    new_string = ""
    for char in string:
        if char in decoder:
            new_letter = decoder.get(char)
        else:
            new_letter =char
        # Now we append the letter to the back of the new string
        new_string = new_string + new_letter
    return new_string
    


In [50]:
decode_text('How are you doing here',random_decoder())

'Hyo sul cyk tyghx alul'

In [51]:
np.math.factorial(26)

403291461126605635584000000L

In [116]:
def generate_proposed_decoder(decoder):
    """
    Define the function generate_proposed_decoder which takes an existing decoder 
    and returns a new decoder as follows: it randomly picks two different letters of the alphabet and swaps the decoding for those letters.
    Make sure to use decoder_letters rather than allowable_letters because we want to keep the spaces as is.
    For example, if we have the decoder {'a':'b','b':'c','c':a'} and decide to swap a and b, our new decoder should be {'a':'c','b':'b','c':a'}
    """
    new_decoder = decoder.copy()
    letters = np.random.choice( list(new_decoder.keys()), 2, replace=False)
    #print(letters)
    letter1,letter2 =letters[0],letters[1]
    new_value_of_letter1,new_value_of_letter2  =new_decoder[letter2],new_decoder[letter1]
    new_decoder[letter1],new_decoder[letter2] = new_value_of_letter1,new_value_of_letter2
    return new_decoder    

In [59]:
l={'a':'b','b':'c','c':'a'}
generate_proposed_decoder(l)

['a' 'b']


{'a': 'c', 'b': 'b', 'c': 'a'}

In [115]:
def log_prob_path(string, wp_bigrams):
    """
    Returns the log of the probability of the path, conditional on the starting character.
    string: the string needs to calculate the log trans score
    wp_bigrams: dictionary of the trans info, key:(s,e) val:trans score of s->e
    Retrun: log prob score of path 
    """
    n = len(string)
    if n<2:
        return 0 
    score = 1 
    for i in range(1,n):
        pre,cur = string[i-1],string[i]
        if (pre,cur) in wp_bigrams:
            #print("Pre:"+pre+" Cur:"+cur)
            score += np.log(wp_bigrams[(pre,cur)])
        else:
            #print(pre,cur)
            print("Not in bigrams")
            
    return score
            

In [72]:
log_prob_path("how are you doing", wp_bigrams)

-39.25140085930973

In [83]:
def log_score(string, decoder, bigrams):
    """
    Define a function log_score that takes the following arguments,
    The function should decode the encrypted string using the decoder,
    and return the log score of the decoder.
    string: an encrypted string
    decoder: a decoder
    bigram: a bigram transition matrix
    return: log score of the dicoder
    """
    t = decode_text(string,decoder)
    return log_prob_path(t, bigrams)

In [86]:
log_score("how are you",random_decoder(),wp_bigrams)

-34.81605218295789

In [87]:
def p_coin(p):
    """
    Flips a coin that comes up heads with probability p and returns 1 if Heads, 0 if Tails
    """
    return np.random.random() < p


In [93]:
p_coin(0.5)

True

In [143]:
def metropolis(string_to_decode, bigrams,reps):
    # Starting decoder
    decoder = random_decoder()
    best_decoder = decoder
    best_score = log_score(string_to_decode,best_decoder,bigrams)
    last_score = best_score
    #print(best_score)
    for rep in np.arange(reps):
        # This will print out our progress
        if rep*10%reps == 0:
            decoded_text = decode_text(string_to_decode,best_decoder)[:40]
            print('Score: %.00f \t Guess: %s'%(best_score, decoded_text))
            
        proposed_decoder = generate_proposed_decoder(best_decoder)
        log_s_orig = log_score(string_to_decode,best_decoder,bigrams)
        log_s_new  = log_score(string_to_decode,proposed_decoder,bigrams)
        # If better than before or p-coin flip works
        if log_s_new > log_s_orig or p_coin(log_s_orig/log_s_new):
            decoder=proposed_decoder
            last_score=log_s_new
            # update the best_score if we had got a higher score 
            if log_s_new > best_score:
                #print("Better score"+str(log_s_new))
                best_decoder=proposed_decoder
                best_score=log_s_new
    return best_decoder
    

In [152]:
ogtext = """I have, myself, full confidence that if all do their duty, if nothing is neglected,
         and if the best arrangements are made, as they are being made, we shall prove ourselve
         once again able to defend our Island home, to ride out the storm of war, and to outliv
         the menace of tyranny, if necessary for years, if necessary alone. At any rate, that i
         what we are going to try to do. That is the resolve of His Majestys Government-every m
         of them. That is the will of Parliament and the nation."""

ogtext2 = """
Even though large tracts of Europe and many old and famous States have fallen or may fall into the grip of the Gestapo and all the odious apparatus of Nazi rule, we shall not flag or fail. We shall go on to the end. We shall fight in France, we shall fight on the seas and oceans, we shall fight with growing confidence and growing strength in the air, we shall defend our island, whatever the cost may be. We shall fight on the beaches, we shall fight on the landing grounds, we shall fight in the fields and in the streets, we shall fight in the hills; we shall never surrender, and if, which I do not for a moment believe, this island or a large part of it were subjugated and starving, then our Empire beyond the seas, armed and guarded by the British Fleet, would carry on the struggle, until, in God's good time, the New World, with all its power and might, steps forth to the rescue and the liberation of the old.
"""

In [153]:
original_string = clean_string(ogtext2)
original_string

'even though large tracts of europe and many old and famous states have fallen or may fall into the grip of the gestapo and all the odious apparatus of nazi rule we shall not flag or fail we shall go on to the end we shall fight in france we shall fight on the seas and oceans we shall fight with growing confidence and growing strength in the air we shall defend our island whatever the cost may be we shall fight on the beaches we shall fight on the landing grounds we shall fight in the fields and in the streets we shall fight in the hills we shall never surrender and if which i do not for a moment believe this island or a large part of it were subjugated and starving then our empire beyond the seas armed and guarded by the british fleet would carry on the struggle until in gods good time the new world with all its power and might steps forth to the rescue and the liberation of the old'

In [154]:
encoding_cipher = random_decoder()
string_to_decode = decode_text(original_string,encoding_cipher)

In [131]:
def encode(string,encoder):
    original_string = clean_string(string)
    return decode_text(original_string,encoding_cipher)

In [164]:
wp_bigrams[('d','d')]

0.01101224337312571

In [155]:
new_decoder = metropolis(string_to_decode, wp_bigrams, 100000)

Score: -4714 	 Guess: ldls cjztgj nxhgl chxwcb zo lthzql xsy p
Score: -2435 	 Guess: even orsudr limde omigot sw eumsfe iny p
Score: -2435 	 Guess: even orsudr limde omigot sw eumsfe iny p
Score: -2435 	 Guess: even orsudr limde omigot sw eumsfe iny p
Score: -2435 	 Guess: even orsudr limde omigot sw eumsfe iny p
Score: -2435 	 Guess: even orsudr limde omigot sw eumsfe iny p
Score: -2435 	 Guess: even orsudr limde omigot sw eumsfe iny p
Score: -2435 	 Guess: even orsudr limde omigot sw eumsfe iny p
Score: -2435 	 Guess: even orsudr limde omigot sw eumsfe iny p
Score: -2435 	 Guess: even orsudr limde omigot sw eumsfe iny p


In [137]:
s = "The text you are going to first encrypt and then decode is taken from Winston Churchill’s speech to the House of Commons on 4 June 1940. Naturally, it contains upper case letters and punctuation that we have not included in our alphabet. In the cell below, the function clean_string strips out all the punctuation and replaces upper case letters by the corresponding lower case letters."
s_e = encode(s,random_decoder())
s_e

'zrc zcaz uji ntc ejfme zj sftgz cmotuyz nmk zrcm kcojkc fg znhcm stjq pfmgzjm oritorfllg gyccor zj zrc rjigc js ojqqjmg jm  ximc  mnzitnllu fz ojmznfmg iyyct ongc lczzctg nmk yimozinzfjm zrnz pc rnwc mjz fmolikck fm jit nlyrnvcz fm zrc ocll vcljp zrc simozfjm olcnmgztfme gztfyg jiz nll zrc yimozinzfjm nmk tcylnocg iyyct ongc lczzctg vu zrc ojttcgyjmkfme ljpct ongc lczzctg'

In [141]:
metropolis(s_e, wp_bigrams, 100000)

Score: -1673 	 Guess: ant atxa jzp uit wzqgw az vqima tgsijya 
Score: -1091 	 Guess: ore oeco yaw ine mausm oa kunto espnyfo 
Score: -1091 	 Guess: ore oeco yaw ine mausm oa kunto espnyfo 
Score: -1091 	 Guess: ore oeco yaw ine mausm oa kunto espnyfo 
Score: -1091 	 Guess: ore oeco yaw ine mausm oa kunto espnyfo 
Score: -1091 	 Guess: ore oeco yaw ine mausm oa kunto espnyfo 
Score: -1091 	 Guess: ore oeco yaw ine mausm oa kunto espnyfo 
Score: -1091 	 Guess: ore oeco yaw ine mausm oa kunto espnyfo 
Score: -1091 	 Guess: ore oeco yaw ine mausm oa kunto espnyfo 
Score: -1091 	 Guess: ore oeco yaw ine mausm oa kunto espnyfo 


{'a': 'c',
 'b': 'x',
 'c': 'e',
 'd': 'q',
 'e': 'm',
 'f': 'u',
 'g': 't',
 'h': 'v',
 'i': 'w',
 'j': 'a',
 'k': 'h',
 'l': 'l',
 'm': 's',
 'n': 'i',
 'o': 'p',
 'p': 'd',
 'q': 'g',
 'r': 'r',
 's': 'k',
 't': 'n',
 'u': 'y',
 'v': 'b',
 'w': 'z',
 'x': 'j',
 'y': 'f',
 'z': 'o'}